These are the two examples from the presentation, runnable online via the magnificent [TLA+ kernel for Jupyter by Stas Kelvich](https://github.com/kelvich/tlaplus_jupyter).

For additional details and the full presentation, please check the accompanying repository.

## A very simple example, finite state machine

This is a very simple, finite, state machine in TLA+ You may recognise the tune

In [12]:
------------------------------- MODULE ThemeSong -------------------------------

EXTENDS Integers

VARIABLES s, count

vars == <<s, count>>

Sinit == s = "na" /\ count = 2

Na == s' = "na" /\ count' = count - 1
ToB == s' = "Batman" /\ count' = 0

Snext == IF count > 0 THEN /\ ( Na \/ ToB)
                      ELSE /\ ToB

Spec == Sinit /\ [][Snext]_vars /\ WF_vars(ToB)

Batman == s = "Batman" /\ count = 0

Liveness == <>Batman

=============================================================================


In [13]:
%tlc:ThemeSong
SPECIFICATION Spec
PROPERTIES Liveness

TLC2 Version 2.14 of 10 July 2019 (rev: 0cae24f)
Running breadth-first search Model-Checking with fp 70 and seed 8061182049641924230 with 4 workers on 4 cores with 3641MB heap and 64MB offheap memory (Mac OS X 10.14.6 x86_64, Oracle Corporation 1.8.0_192 x86_64, MSBDiskFPSet, DiskStateQueue).
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/tmpi73fvaod/ThemeSong.tla
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/Integers.tla
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/Naturals.tla
Semantic processing of module Naturals
Semantic processing of module Integers
Semantic processing of module ThemeSong
Starting... (2020-01-06 20:58:38)
Implied-temporal checking--satisfiability problem has 1 branches.
Computing initial states...
Finished computing initial states: 1 distinct state generated at 2020-01-06 20:58:38.
Progress(3) at 2020-01-06 20:58:38: 7 states generated, 4 distinct states found, 0 states left on queue.
Checkin

Below, we rewrite it as Pluscal "P" variant, and get the auto-generated TLA code

In [ ]:
---------------------------- MODULE ThemeSongPC ----------------------------

EXTENDS Integers

(*--fair algorithm ThemeSong
    variables count = 5, s = "na"
    begin ThemeSong:
        while count > 0 do
            either
                s := "Batman";
                count := 0;
            or
                s := "na";
                count := count - 1;
            end either;
        end while;
        if count = 0 then
            s := "Batman";
        end if;
    end algorithm; -*)

\* BEGIN TRANSLATION
VARIABLES count, s, pc

vars == << count, s, pc >>

Init == (* Global variables *)
        /\ count = 5
        /\ s = "na"
        /\ pc = "ThemeSong"

ThemeSong == /\ pc = "ThemeSong"
             /\ IF count > 0
                   THEN /\ \/ /\ s' = "Batman"
                              /\ count' = 0
                           \/ /\ s' = "na"
                              /\ count' = count - 1
                        /\ pc' = "ThemeSong"
                   ELSE /\ IF count = 0
                              THEN /\ s' = "Batman"
                              ELSE /\ TRUE
                                   /\ s' = s
                        /\ pc' = "Done"
                        /\ count' = count

(* Allow infinite stuttering to prevent deadlock on termination. *)
Terminating == pc = "Done" /\ UNCHANGED vars

Next == ThemeSong
           \/ Terminating

Spec == /\ Init /\ [][Next]_vars
        /\ WF_vars(Next)

Termination == <>(pc = "Done")

\* END TRANSLATION

Batman == s = "Batman" /\ count = 0

Liveness == <>Batman


=============================================================================


In [15]:
%tlc:ThemeSongPC
SPECIFICATION Spec
PROPERTIES Liveness

TLC2 Version 2.14 of 10 July 2019 (rev: 0cae24f)
Running breadth-first search Model-Checking with fp 122 and seed 7166269695903366348 with 4 workers on 4 cores with 3641MB heap and 64MB offheap memory (Mac OS X 10.14.6 x86_64, Oracle Corporation 1.8.0_192 x86_64, MSBDiskFPSet, DiskStateQueue).
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/tmpp6tpplur/ThemeSongPC.tla
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/Integers.tla
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/Naturals.tla
Semantic processing of module Naturals
Semantic processing of module Integers
Semantic processing of module ThemeSongPC
Starting... (2020-01-06 20:58:40)
Implied-temporal checking--satisfiability problem has 1 branches.
Computing initial states...
Finished computing initial states: 1 distinct state generated at 2020-01-06 20:58:41.
Progress(6) at 2020-01-06 20:58:41: 14 states generated, 8 distinct states found, 0 states left on queue.
C

## A very simple actor model example

A very simple (Akka based) actor that on a message triggers a third party would look like the following in Scala

```scala
case class Trigger(payload: Payload)

class SimpleActor extends Actor {
  def receive = {
    case Trigger(payload) => ThirdParty().trigger(payload)
    case _                => log.warn("Duh")
  }
}
```

We need to simulate the inboxes that the actor system offers, and simulate the mailbox by the actor fetching messages itself. Below you can see the PlusCal "C" code, followed by the TLA+ automated translation

In [ ]:
------------------------------- MODULE Actor -------------------------------

EXTENDS TLC, Integers, Sequences

(*--fair algorithm ActorStuff {

variables actorInboxes = [ actor |-> <<>> ];
          triggered = FALSE;

procedure trigger(trigger_content="?") {
    triggerLabel:
      triggered := TRUE;
      return;
}

fair process (actor = "actor")
variables currentMessage = <<"?", "no_content">>;
  kind = "?";
  content = "no_content";
{
  Send:
    actorInboxes["actor"] := Append(actorInboxes["actor"], <<"trigger", "foo">>);
  WaitForMessages:+
    while (TRUE) {
      if (actorInboxes["actor"] /= <<>>) {
         currentMessage := Head(actorInboxes["actor"]);               
         content := Head(Tail(currentMessage));
         kind := Head(currentMessage);
         actorInboxes["actor"] := Tail(actorInboxes["actor"]);
        };
        ProcessMessage:
         if (kind = "trigger") {
           call trigger(content);
         }
    }
}

}
*)
\* BEGIN TRANSLATION
VARIABLES actorInboxes, triggered, pc, stack, trigger_content, currentMessage, 
          kind, content

vars == << actorInboxes, triggered, pc, stack, trigger_content, 
           currentMessage, kind, content >>

ProcSet == {"actor"}

Init == (* Global variables *)
        /\ actorInboxes = [ actor |-> <<>> ]
        /\ triggered = FALSE
        (* Procedure trigger *)
        /\ trigger_content = [ self \in ProcSet |-> "?"]
        (* Process actor *)
        /\ currentMessage = <<"?", "no_content">>
        /\ kind = "?"
        /\ content = "no_content"
        /\ stack = [self \in ProcSet |-> << >>]
        /\ pc = [self \in ProcSet |-> "Send"]

triggerLabel(self) == /\ pc[self] = "triggerLabel"
                      /\ triggered' = TRUE
                      /\ pc' = [pc EXCEPT ![self] = Head(stack[self]).pc]
                      /\ trigger_content' = [trigger_content EXCEPT ![self] = Head(stack[self]).trigger_content]
                      /\ stack' = [stack EXCEPT ![self] = Tail(stack[self])]
                      /\ UNCHANGED << actorInboxes, currentMessage, kind, 
                                      content >>

trigger(self) == triggerLabel(self)

Send == /\ pc["actor"] = "Send"
        /\ actorInboxes' = [actorInboxes EXCEPT !["actor"] = Append(actorInboxes["actor"], <<"trigger", "foo">>)]
        /\ pc' = [pc EXCEPT !["actor"] = "WaitForMessages"]
        /\ UNCHANGED << triggered, stack, trigger_content, currentMessage, 
                        kind, content >>

WaitForMessages == /\ pc["actor"] = "WaitForMessages"
                   /\ IF actorInboxes["actor"] /= <<>>
                         THEN /\ currentMessage' = Head(actorInboxes["actor"])
                              /\ content' = Head(Tail(currentMessage'))
                              /\ kind' = Head(currentMessage')
                              /\ actorInboxes' = [actorInboxes EXCEPT !["actor"] = Tail(actorInboxes["actor"])]
                         ELSE /\ TRUE
                              /\ UNCHANGED << actorInboxes, currentMessage, 
                                              kind, content >>
                   /\ pc' = [pc EXCEPT !["actor"] = "ProcessMessage"]
                   /\ UNCHANGED << triggered, stack, trigger_content >>

ProcessMessage == /\ pc["actor"] = "ProcessMessage"
                  /\ IF kind = "trigger"
                        THEN /\ /\ stack' = [stack EXCEPT !["actor"] = << [ procedure |->  "trigger",
                                                                            pc        |->  "WaitForMessages",
                                                                            trigger_content |->  trigger_content["actor"] ] >>
                                                                        \o stack["actor"]]
                                /\ trigger_content' = [trigger_content EXCEPT !["actor"] = content]
                             /\ pc' = [pc EXCEPT !["actor"] = "triggerLabel"]
                        ELSE /\ pc' = [pc EXCEPT !["actor"] = "WaitForMessages"]
                             /\ UNCHANGED << stack, trigger_content >>
                  /\ UNCHANGED << actorInboxes, triggered, currentMessage, 
                                  kind, content >>

actor == Send \/ WaitForMessages \/ ProcessMessage

(* Allow infinite stuttering to prevent deadlock on termination. *)
Terminating == /\ \A self \in ProcSet: pc[self] = "Done"
               /\ UNCHANGED vars

Next == actor
           \/ (\E self \in ProcSet: trigger(self))
           \/ Terminating

Spec == /\ Init /\ [][Next]_vars
        /\ WF_vars(Next)
        /\ WF_vars(actor) /\ SF_vars(WaitForMessages) /\ WF_vars(trigger("actor"))

Termination == <>(\A self \in ProcSet: pc[self] = "Done")

\* END TRANSLATION

Triggered == triggered = TRUE

Liveness == <>Triggered

=============================================================================


And we check that eventually the third party is triggered, in the following specification property check

In [17]:
%tlc:Actor
SPECIFICATION Spec
PROPERTY Liveness

TLC2 Version 2.14 of 10 July 2019 (rev: 0cae24f)
Running breadth-first search Model-Checking with fp 121 and seed -2385201558344987953 with 4 workers on 4 cores with 3641MB heap and 64MB offheap memory (Mac OS X 10.14.6 x86_64, Oracle Corporation 1.8.0_192 x86_64, MSBDiskFPSet, DiskStateQueue).
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/tmp6u3mw1hj/Actor.tla
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/TLC.tla
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/Integers.tla
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/Sequences.tla
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/Naturals.tla
Parsing file /private/var/folders/dh/0s4sld5s23x9mvcl87x1y6_w0000gn/T/FiniteSets.tla
Semantic processing of module Naturals
Semantic processing of module Sequences
Semantic processing of module FiniteSets
Semantic processing of module TLC
Semantic processing of module Integers
Semant